In [11]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.spatial import distance


In [3]:
img = cv2.imread('data/R100/R100f_big5.jpg',cv2.IMREAD_COLOR)
img_blur = cv2.GaussianBlur(img, (5,5), 0)
img_gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
img_eq = cv2.equalizeHist(img_gray)
img_neg = cv2.bitwise_not(img_eq)
img_resized = cv2.resize(img_neg, (574,265))







In [4]:
edges = cv2.Canny(img_resized, 100, 200)
_, thresh = cv2.threshold(img_resized, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

kernel = np.ones((5,5), np.uint8)
dilation = cv2.dilate(thresh, kernel, iterations=1)
erosion = cv2.erode(dilation, kernel, iterations=1)

contours, _ = cv2.findContours(erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
img_contours = cv2.drawContours(img_resized, contours, -1, (0,255,0), 3)

#display images
cv2.imshow('Edges Image', edges)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
#orb feature extraction on edges
orb = cv2.ORB_create()
keypoints, descriptors = orb.detectAndCompute(edges, None)
# show image with keypoints and descriptors
img_keypoints = cv2.drawKeypoints(edges, keypoints, None)
cv2.imshow('ORB keypoints', img_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [77]:
#print grayed image
cv2.imshow('gray', img_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
data_dir = 'data'
classes = ['R10', 'R20', 'R50', 'R100', 'R200']

orb = cv2.ORB_create()

features = []
labels = []

def preprocess_and_extract_features(img_path, max_features=500):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    
    img_blur = cv2.GaussianBlur(img, (5,5), 0)
    img_gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    img_neg = cv2.bitwise_not(img_eq)
    img_resized = cv2.resize(img_neg, (574,265))
    edges = cv2.Canny(img_resized, 100, 200)

    keypoints, descriptors = orb.detectAndCompute(edges, None)
    if descriptors is None:
        descriptors = np.zeros((max_features, orb.descriptorSize()), dtype=np.uint8)
    else:
        # Pad or truncate descriptors to ensure uniform length
        if len(descriptors) < max_features:
            pad_length = max_features - len(descriptors)
            descriptors = np.pad(descriptors, ((0, pad_length), (0, 0)), 'constant')
        elif len(descriptors) > max_features:
            descriptors = descriptors[:max_features]
    return descriptors

for label in classes:
    class_dir = os.path.join(data_dir, label)
    for filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, filename)
        descriptors = preprocess_and_extract_features(img_path)

        for descriptor in descriptors:
            features.append(descriptor.ravel())
            labels.append(label)

features = np.array(features)
labels = np.array(labels)

# Manual KNN Classifier
def knn_classify(X_train, y_train, x_test, k=3):
    distances = []
    for i in range(len(X_train)):
        dist = distance.euclidean(X_train[i], x_test)
        distances.append((dist, y_train[i]))
    distances.sort(key=lambda x: x[0])
    neighbors = distances[:k]
    class_votes = {}
    for neighbor in neighbors:
        label = neighbor[1]
        class_votes[label] = class_votes.get(label, 0) + 1
    sorted_votes = sorted(class_votes.items(), key=lambda x: x[1], reverse=True)
    return sorted_votes[0][0]

# Function to classify a new input image
def classify_new_image(img_path, X_train, y_train, k=3):
    descriptors = preprocess_and_extract_features(img_path)
    # Flatten descriptors to match training data structure
    descriptors = descriptors.reshape(-1, orb.descriptorSize())
    predictions = []
    for descriptor in descriptors:
        prediction = knn_classify(X_train, y_train, descriptor, k)
        predictions.append(prediction)
    return max(set(predictions), key=predictions.count)

# Test with a new input image
test_img_path = 'test200.jpg'
prediction = classify_new_image(test_img_path, features, labels)
print(f'The predicted class for the input image is: {prediction}')

The predicted class for the input image is: R100


In [7]:
import cv2
import numpy as np
import os
from scipy.spatial import distance

# Define paths and classes
data_dir = 'data'
classes = ['R10', 'R20', 'R50', 'R100', 'R200']

# Initialize ORB detector
orb = cv2.ORB_create()

# Initialize lists to hold features and labels
features = []
labels = []

def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)
    img_gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    img_neg = cv2.bitwise_not(img_eq)
    img_resized = cv2.resize(img_neg, (574, 265))
    edges = cv2.Canny(img_resized, 100, 200)
    return edges

def extract_features(img, max_features=500):
    keypoints, descriptors = orb.detectAndCompute(img, None)
    if descriptors is None:
        descriptors = np.zeros((max_features, orb.descriptorSize()), dtype=np.uint8)
    else:
        # Pad or truncate descriptors to ensure uniform length
        if len(descriptors) < max_features:
            pad_length = max_features - len(descriptors)
            descriptors = np.pad(descriptors, ((0, pad_length), (0, 0)), 'constant')
        elif len(descriptors) > max_features:
            descriptors = descriptors[:max_features]
    return descriptors

# Process all images and extract features
for label in classes:
    class_dir = os.path.join(data_dir, label)
    for filename in os.listdir(class_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(class_dir, filename)
            preprocessed_img = preprocess_image(img_path)
            descriptors = extract_features(preprocessed_img)
            
            for descriptor in descriptors:
                features.append(descriptor.ravel())
                labels.append(label)

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Manual KNN Classifier
def knn_classify(X_train, y_train, x_test, k=5):
    distances = []
    for i in range(len(X_train)):
        dist = distance.euclidean(X_train[i], x_test)
        distances.append((dist, y_train[i]))
    distances.sort(key=lambda x: x[0])
    neighbors = distances[:k]
    class_votes = {}
    for neighbor in neighbors:
        label = neighbor[1]
        class_votes[label] = class_votes.get(label, 0) + 1
    sorted_votes = sorted(class_votes.items(), key=lambda x: x[1], reverse=True)
    return sorted_votes[0][0]

# Function to classify a new input image
def classify_new_image(img_path, X_train, y_train, k=5):
    preprocessed_img = preprocess_image(img_path)
    descriptors = extract_features(preprocessed_img)
    # Flatten descriptors to match training data structure
    descriptors = descriptors.reshape(-1, orb.descriptorSize())
    predictions = []
    for descriptor in descriptors:
        prediction = knn_classify(X_train, y_train, descriptor, k)
        predictions.append(prediction)
    return max(set(predictions), key=predictions.count)

# Test with a new input image
test_img_path = 'test50.jpg'
prediction = classify_new_image(test_img_path, features, labels)
print(f'The predicted class for the input image is: {prediction}')


The predicted class for the input image is: R200
